In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
path = Path('/app/data/raw/2022.03.22OGW.xlsx')
assert path.exists()

In [ ]:
# eip = dbcp.extract.eip_infrastructure.extract(path)
# hardcode the extract function so this notebook can be easily rerun in the future without maintenance
sheets_to_read = [
    'Facility',
    'Company',
    'Project',
    'Air Construction',  # permit status is key to identifying actionable projects
    'Pipelines',
]
eip = pd.read_excel(path, sheet_name=sheets_to_read)
rename_dict = {
    'Facility': 'eip_facilities',
    'Company': 'eip_companies',
    'Project': 'eip_projects',
    'Air Construction': 'eip_air_constr_permits',
    'Pipelines': 'eip_pipelines',
}
eip = {rename_dict[key]: df for key, df in eip.items()}

In [ ]:
eip.keys()

In [ ]:
{k: df.shape for k, df in eip.items()}

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
fac = eip['eip_facilities']
cos = eip['eip_companies']
proj = eip['eip_projects']
air = eip['eip_air_constr_permits']
pipe = eip['eip_pipelines']

Outline of work
Two parts: data cleaning and data normalization/structuring
# Structuring and Normalizaing
**5 entities and 5 many:many relationships means 10 tables...** But utting both pipelines and companies drops the total tables in half to 5.

The *only* purpose of bringing in the companies table is to add one column with ownership info. But the marginal cost is 3 tables (2 if cutting pipelines), or 30% of tables. I'll confirm with DBCP that this is OK.

Pipelines are approved at the federal level so I'm not sure they are actionable for Down Ballot people. They also have only very coarse location information (state). We punted on them last time so I would like to do so again. Marginal cost is also 3 tables, or 2 additional if cutting companies.

## Entity Relationships
### Entities
* facilities
* companies
* projects
* permits (air construction permits. there are many other permit types that I didn't integrate)
* pipelines

### Relationships
many : many
* facilities : companies
* facilities : projects
* facilities : pipelines
* companies : pipelines
* projects : permits

one : many
* none

one : one
* none

no direct relationship
* facilities : permits (air construction permits are mediated through projects. Other permits not considered here do have direct relationships)
* companies : projects (mediated through facilities)
* companies : permits (mediated through projects then through facilities)
* projects : pipelines (mediated through facilities)
* permits : pipelines (mediated through projects then through facilities)

# Cleaning
Need to clean facilities, projects, and permits via the usual checklist. But I can ignore many unecessary columns and prefix them 'raw_' to discourage use.
## Facilities Cleaning
- [x] Accuracy
- [x] Atomicity
- [ ] Consistency
- [x] Completeness
- [x] Uniformity
- [x] Validity
    - [x] Range Validation
    - [x] Uniqueness Validation
    - [x] Set Membership Validation
    - [x] Type Validation
    - [x] Cross-Field Validation

### Accuracy
I'm mostly using this table for location information, so I'll focus on the "street address" and "coordinates" columns. I don't have "golden data" to compare against, but I can at least spot check some items by googling them. \[Update: 3/3 spot checks of location are good. Obviously this is far from comprehensive but gives a small measure of confidence.]

In [ ]:
fac.sample(3, random_state=42)

Googling "Oak Grove Gas Plant" turns up the facility. [Street address](https://www.google.com/maps/place/Williams+Natural+Gas+Oak+Grove+Facility/@39.871189,-80.6944623,1177m/data=!3m1!1e3!4m13!1m7!3m6!1s0x0:0x6769abd010d373f9!2zMznCsDUyJzMyLjkiTiA4MMKwNDEnNDUuMiJX!3b1!8m2!3d39.8758!4d-80.6959!3m4!1s0x8835e69402fb74cd:0x94b44b7720f51c5!8m2!3d39.8690544!4d-80.693195) and coordinates match. Owner also matches.

MarkWest Houston Complex location is also good. Google maps labels the [corporate office](https://www.google.com/maps/place/MarkWest+Houston+Plant/@40.262237,-80.2596898,1240m/data=!3m1!1e3!4m13!1m7!3m6!1s0x8834528cbcacb571:0xbd8b49797f3fdd4!2s800+Western+Ave,+Washington,+PA+15301!3b1!8m2!3d40.2584361!4d-80.2555021!3m4!1s0x8834539d500f0e45:0x248d758337e3de37!8m2!3d40.2585062!4d-80.254957) as across the street from the given address, which belongs to a different facility building. But that doesn't matter for our purposes -- we aren't sending them a letter. Owner also matches.

Formosa Point Comfort plant street address matches [google maps](https://www.google.com/maps/place/Formosa+Plastics+Corporation,+Texas/@28.6804226,-96.5626898,13964m/data=!3m1!1e3!4m5!3m4!1s0x0:0x469e4fbb5f6d12a1!8m2!3d28.6975144!4d-96.5449333) and coordinates are inside the facility. Owner also matches.

### Atomicity
By inspection I see that all the ID and associated name fields can contain multiple values: company, project, pipelines, and permits. The location fields are mercifully single valued

In [ ]:
# street address does not look multi-valued but has other problems. Thankfully lat lon is still available
# a little more digging suggests bad addresses are because these have not yet been built.
# Can't check for sure until I can join project status on to facilities
pd.options.display.max_colwidth = 0
fac.loc[fac['Street Address'].str.len().nlargest(10).index, ['id', 'name', 'Street Address', 'Location']]

In [ ]:
# location is not multi-valued - exactly two decimal points per coordinate pair
fac['Location'].str.count('\.').agg(['min', 'max'])

In [ ]:
# a shitload of missing facility IDs, but no multi-valued ones
fac['Facility ID'].describe()

### Completeness
Notable missing values and lack of missing values:
* 93/563 (16.5%) missing street address. Plus some addresses are not missing but look unusable.
* 4/563 (0.7%) of facilities are missing linked Project IDs
* 9/563 (1.6%) missing "Location" (coordinates)
* 3/563 (tiny%) missing county (none missing state). But the true test is how successful `addfips` is with these pairs
* 60 to 100 (10% to 18%) missing EJ Screen metrics, depending on which metric

I don't know what `Facility ID` is (vs `id` of this facility table), but 179/563 (31.8%) rows are missing `Facility ID`. They have different numerical ranges and I see that the companies and project tables thankfully use the `id` numbers, which are 100% complete.

Based on these nan counts, I should first try `addfips` on state/county pairs. If too many fail, the most complete option is to geocode via coordinates.

In [ ]:
len(fac)

In [ ]:
fac.count().T

### Consistency - defer
Defer until I've cleaned the related datasets
### Uniformity
Important columns to check consistent representation:
* coordinates
* ID fields (check consistent array delimiters)

Secondary importance:
* street address (this is a luxury field)
* modified_on

#### Coordinates

In [ ]:
# "-XX.X, YY.Y" with 2 or 3 digits before the decimal and 2 to 7 digits after.
# Plus optional leading/trailing whitespace.
coord_pattern = r'\s*-\d{2,3}\.\d{2,7}, \d{2,3}\.\d{2,7}\s*'
fac['Location'].str.match(coord_pattern).sum()

In [ ]:
# matches count, so they all have the same formatting
fac['Location'].count()

In [ ]:
# tighten criteria to 3+ digits after decimal
# Reveals that only 2 facilities have poor precision (plus or minus about a km)
coord_pattern = r'\s*-\d{2,3}\.\d{3,7}, \d{2,3}\.\d{3,7}\s*'
fac['Location'].str.match(coord_pattern).sum()

#### ID Fields
Want to check for consistent array delimiters.

In [ ]:
# exclude ID cols with numeric types (no arrays present)
id_cols = [col for col in fac.columns if '(ID)' in col and pd.api.types.is_object_dtype(fac[col])]
id_cols

In [ ]:
# mandatory opening pattern, optional delimiter, optional repeating pattern, optional closing pattern, mandatory end of line
array_pattern = r'(?:\d{3,5})(?:, ?)?(?:\d{3,5}, ?)*(?:\d{3,5})?$'

In [ ]:
test_case = pd.Series([
    '1234',
    '1234,567',
    '1234, 567',
    '12345, 678, 9012',
    '1234\t5678', # tab is bad, no comma
    '12, 3456', # too short
    '1234    5678', # too many spaces, no comma
])
pd.concat([test_case, test_case.str.match(array_pattern)], axis=1)

In [ ]:
# all pass the formatting test
for col in id_cols:
    assert fac[col].str.match(array_pattern).all()

#### Date Modified

In [ ]:
# to_datetime works on all values present
timestamps = pd.to_datetime(fac['modified_on'])
timestamps.dtypes, timestamps.isna().sum()

#### Street Address - defer
hard to test and I don't care that much if it's wrong. Best way to test is probably to outsource to a pre-built geocoder

### Range Validation
Check IDs and Coordinates
#### Coordinates
All the extreme coordinates are real places! No "Null Island" dwellers either.

In [ ]:
coords = fac['Location'].str.split(',', expand=True)
for col in coords.columns:
    coords.loc[:, col] = pd.to_numeric(coords.loc[:, col], errors='coerce')
coords.head()

In [ ]:
coords.describe()

In [ ]:
# look at extreme coordinates
# max longitude
fac.loc[coords[0].idxmax(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

In [ ]:
# Min longitude
fac.loc[coords[0].idxmin(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

In [ ]:
# max latitude
fac.loc[coords[1].idxmax(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

In [ ]:
# Min Latitude
fac.loc[coords[1].idxmin(), ['Location', 'City', 'ZIP Code', 'Facility Description']]

#### IDs
There are lots of ID columns, but I only care about project IDs and associated facilities

In [ ]:
# defined way up near the top
id_cols

In [ ]:
proj_ids = fac['Project (ID)'].str.split(',', expand=True)
for col in proj_ids.columns:
    proj_ids.loc[:, col] = pd.to_numeric(proj_ids.loc[:, col], errors='coerce')

proj_ids.head()

In [ ]:
# they all look in the same range
proj_ids.describe()

In [ ]:
assoc_ids = fac['Associated Facilities (ID)'].str.split(',', expand=True)
for col in assoc_ids.columns:
    assoc_ids.loc[:, col] = pd.to_numeric(assoc_ids.loc[:, col], errors='coerce')

assoc_ids.head()

In [ ]:
# they all look in the same range
assoc_ids.describe()

### Uniqueness Validation
Check the `id` field (NOT `Facility ID`)

In [ ]:
fac['id'].duplicated().sum()

### Set Membership Validation
Check state/county only. A few takeaways:
* state 'TDB' values need conversion to NULL
* a few states are arrays (but only one value, duplicated)
* 4 counties are arrays. Probably just take the first one. The better but more expensive way is to use the given lat, lon coords. Not worth it for 4 facilities.

In [ ]:
from pudl.helpers import add_fips_ids

In [ ]:
w_fips = add_fips_ids(fac[['State', 'County or Parish']], state_col='State', county_col='County or Parish', vintage=2020)
w_fips.head()

In [ ]:
w_fips.shape

In [ ]:
# 9 bad states and 20 bad state/county combos
w_fips.describe()

In [ ]:
w_fips[w_fips['county_id_fips'].isna()]

### Type Validation
All the ID columns and the coordinates are CSV string arrays that need parsing and conversion to numeric.
### Cross-Field Validation - Defer
A thorough cleaning would involve geocoding the given coordinates and making sure they match the given state, county values. Also reverse geocoding the given street address and computing distance vs given coordinates. But I'll defer that until we actually do something with the lat, lon values.